In [5]:
import pandas as pd
import pubchempy as pcp
from pybel import readstring
import requests
import time
from pychem.pychem import *
from rdkit import Chem
from StringIO import StringIO
import dill as pickle

ImportError: No module named dill

In [2]:
chemicals = pd.read_csv('../data/version-3/food-chemical.tsv', sep='\t')
print("Unique chemicals: ", len(set(chemicals['pubchem-id'])))

('Unique chemicals: ', 6992)


In [3]:
sdf_files = list()
completed = set()

In [4]:
while len(sdf_files) != len(set(chemicals['pubchem-id'])):
    try:
        for i, pubchem in enumerate(set(chemicals['pubchem-id'])):
            if i not in completed:
                r = requests.get('https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/%i/SDF' % pubchem)
                sdf_files.append(readstring('sdf', r.text))
                completed.add(i)

            if i % 100 == 0:
                print("Completed: %i" % len(sdf_files))
    except KeyboardInterrupt:
        break
    except:
        print("Sleeping for a lil time.")
        time.sleep(30)

Completed: 1
Sleeping for a lil time.
Completed: 63
Completed: 101
Completed: 201
Completed: 301
Completed: 501
Completed: 601
Completed: 701
Completed: 801
Completed: 901
Completed: 1001
Completed: 1101
Completed: 1201
Completed: 1301
Completed: 1401
Completed: 1501
Completed: 1601
Completed: 1701
Completed: 1801
Completed: 1901
Completed: 2001
Completed: 2101
Completed: 2201
Completed: 2301
Completed: 2401
Completed: 2501
Completed: 2601
Sleeping for a lil time.
Completed: 2646
Completed: 2646
Completed: 2646
Completed: 2646
Completed: 2646
Completed: 2646
Completed: 2646
Completed: 2646
Completed: 2646
Completed: 2646
Completed: 2646
Completed: 2646
Completed: 2646
Completed: 2646
Completed: 2646
Completed: 2646
Completed: 2646
Completed: 2646
Completed: 2646
Completed: 2646
Completed: 2646
Completed: 2646
Completed: 2646
Completed: 2646
Completed: 2646
Completed: 2646
Completed: 2646
Completed: 2701
Completed: 2801
Sleeping for a lil time.
Completed: 2881
Completed: 2881
Completed:

In [12]:
# Write sdf files to disk.
for f in sdf_files:
    f.write('sdf', filename='../data/version-3/sdf-files/' + f.title + '.sdf')

In [5]:
# Get properties.
def generate_properties(mol):
    props = {}
    try:
        props.update(constitution.GetConstitutional(mol))
        props.update(connectivity.GetConnectivity(mol))
        props.update(kappa.GetKappa(mol))
        props.update(bcut.GetBurden(mol))
        props.update(estate.GetEstate(mol))
        props.update(basak.Getbasak(mol))
        props.update(moran.GetMoranAuto(mol))
        props.update(geary.GetGearyAuto(mol))
        props.update(molproperty.GetMolecularProperty(mol))
        props.update(charge.GetCharge(mol))
        props.update(moe.GetMOE(mol))
    except:
        pass

    return props

properties = list()
details = list()
covered = set()

In [ ]:
for i, mol in enumerate(sdf_files):
    molecule = Chem.MolFromMolBlock(mol.write('sdf'))
    properties.append(generate_properties(molecule))
    details.append(mol.data)
    if i % 100 == 0:
        print("Completed: %i" % i)
        
    break

In [ ]:
details

In [ ]:
props_to_keep = ['Weight', 'nhyd', 'nring', 'nrot', 'ndonr', 'naccr', 'nta',
                 'naro', 'nhev', 'Hy', 'LogP']

# Merge with main dataframe
df = pd.concat([df, props_df[props_to_keep]], 1)

# Remove molecules for which null properties exist.
df = df.loc[props_df.isnull().sum(axis=1) == 0]

df.shape

'101679106\n OpenBabel06101803582D\n\n 45 48  0  0  1  0  0  0  0  0999 V2000\n    5.6405   -3.5716    0.0000 O   0  0  0  0  0  0  0  0  0  0  0  0\n    4.6622    4.1346    0.0000 O   0  0  0  0  0  0  0  0  0  0  0  0\n    6.4361    3.1105    0.0000 O   0  0  0  0  0  0  0  0  0  0  0  0\n    3.8372   -4.6313    0.0000 O   0  0  0  0  0  0  0  0  0  0  0  0\n    2.9180    0.0654    0.0000 N   0  3  0  0  0  0  0  0  0  0  0  0\n    2.0080   -0.4414    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0\n    3.7841   -0.4346    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0\n    2.0000   -1.4830    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0\n    3.8001   -1.4761    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0\n    2.9021   -2.0039    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0\n    2.9180    1.0654    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0\n    4.6501    0.0654    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0\n    3.7841    1.5654    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0